# Apuntes Aprendizaje Reforzado

Fuente: https://www.youtube.com/watch?v=Mut_u40Sqz4

In [4]:
#!pip install stable-baselines3[extra]
#!pip install gym

     ------------------------------------ 721.7/721.7 kB 506.0 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827632 sha256=511e7ee1419bda8ebe23b4917cb5d03d102a4d39851eb2dc5106351674853da6
  Stored in directory: c:\users\jpeir\appdata\local\pip\cache\wheels\17\79\65\7afedc162d858b02708a3b8f7a6dd5b1000dcd5b0f894f7cc1
Successfully built gym


In [5]:
import os
import gym
from stable_baselines3 import PPO
#Esto es el el algoritmo de aprendizaje que vamos a utilizar, aunque hay muchos diferentes
# y puede que este no sea el que nosotros necesitemos
from stable_baselines3.common.vec_env import DummyVecEnv
# DummyVecEnv nos sirve para vectorizar nuestros entornos y es maravilloso porque nos
# posibilita hacer muchas simulaciones en un vector, lo que acelera el proceso de manera extrema
from stable_baselines3.common.evaluation import evaluate_policy
# Esta libraria es para evaluar modelos

## Cargar Entorno

In [6]:
environment_name = "CartPole-v0"
env = gym.make(environment_name)
# Vamos a trabajar con un problema de RL que consiste en mantener una barra vertical sin que caiga
# La barra se encuentra sobre una plataforma móvil
# El problema con el que trabajamos y cómo este se describe es lo que llamamos el entorno

C:\Users\jpeir\anaconda3\lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [26]:
episodes = 5
for ep in range(episodes):
    state = env.reset()
    # Reinicia el entorno
    done = False
    # Estado del entorno
    score = 0
    # Recompensa/Puntuación
    
    while not done:
        env.render()
        # Muestra una interfaz del problema
        action = env.action_space.sample()
        # Realiza una acción aleatoria
        # En este problema, como tal y como se define en el entorno, solo se pueden hacer 2 posibles opciones, izquierda o derecha
        # Así que action_space es el espacio de posibles acciones, que como además son acciones discretas el objeto se guarda como Discrete(2)
        # Luego, cuando hacemos sample tomamos una de las 2 posibles acciones aleatoriamente
        n_state, reward, done, trunc, info = env.step(action)
        # Ejecutamos la accion elegida con step
        # Devuelve el estado al que pasamos (la definición del estado)
        # La recompensa que adquiere en ese paso
        # Si la mision se ha completado
        # Si la mision ha podido completarse en menos del maximo de pasos posibles
        # Informacion adicional
        score += reward
    print("Episode:{} Score:{}".format(ep, score))
#env.close()

Episode:0 Score:9.0
Episode:1 Score:13.0
Episode:2 Score:22.0
Episode:3 Score:20.0
Episode:4 Score:14.0


In [15]:
 env.reset()
env.action_space.sample()
env.step(action)

(array([ 0.0193603 ,  0.21330012, -0.02149993, -0.25753242], dtype=float32),
 1.0,
 False,
 False,
 {})